# Análisis y visualización de datos con python
# 3. Indexación y selección de dataframes

    - a) Estructura de una tabla 
    - b) Slicing
    - c) Únicos y conteo
    - d) Ordenar
    - e) Subsetting
    - f) Guardar datos

---

## 4.a Estructura de una tabla 

En primer lugar cargaremos el Histórico de las Bitácoras Onomásticas (HBO) como vimos en el tutorial anterior. 

Adicionalmente, marcaremos que la columna `ID` es el indice de la tabla usando la opción `index_col`. El indice debé de ser único, es decir, los valores no deben de repetirse. También transformaremos la columna `Edad_transcrito` a datos numéricos, ignorando por ahora los textos.

In [ ]:
import pandas as pd

filename = 'data_raw/CNB_DOB_BPGS_Respuesta_Solicitud 332163723000249.xlsx'
df = pd.read_excel(filename, #nombre del archivo
                   sheet_name="HBO", #nombre de la hoja
                   index_col='ID' #nombre de la columna del indice
                  )
df['Edad_transcrito'] = pd.to_numeric(df['Edad_transcrito'], errors='coerce') #Limpieza
df

Una tabla de pandas o dataframe consta de varios elementos:
* Nombres de columnas, las cuales pueden ser vistas como la primera fila en letras negritas. Para verlas usa _.columns_.
* Nombres de filas, las cuales pueden ser vistas como la primera columna, la cual carece de nombre. Para verlas usa _.index_.
* Valores, los datos dentro de la tabla. Para verlas usa _.values_.
* El número de filas y columnas total. Para verla usa _.shape_.
* Tipo de datos de cada columna. Para verla usa _.dtype_.

Con el fin de conocer el nombre de las columnas usaremos la opción _.columns_. Es importante notar que esta opción no tiene paréntesis no corchetes, ya que es un elemento del objeto.

In [ ]:
df.columns

Estas columnas corresponden a las del diccionario de datos que se encuentra en la Hoja 2 del archivo.

### Ejercicio 1

Usa cada una de las opciones mencionadas en esta sección y describe que regresa.
* `df.index`
* `df.values`
* `df.shape`
* `df.dtypes`


In [ ]:
df.shape

## 4.b Slicing

Muchas veces no queremos trabajar sobre toda la tabla de datos, en su lugar queremos usar unicamente una fracción de ella. Para ello, existen varias formas de selecionar partes de una tabla.

Por ejemplo, la función `.head()` devuelve las primeras filas de la tabla. Por defecto estos comandos muestran solo cinco filas, pero es posible cambiar el número de filas poniendo un número entre paréntesis.

In [ ]:
df.head()

Si queremos ver las últimas filas se puede usar el comando `.tail()`. 

En ambos casos se puede específicar el número de filas que se quieren ver

In [ ]:
df.tail( 10 )

Para ver una columna específica se pone el nombre del dataframe seguido por corchetes con el nombre de la columna. Una sola columna es del tipo `pd.Series`. Por ejemplo, para ver la columna del sexo ponemos su nombre 'Sexo' entre corchetes. Esto es similar a la forma en la que se seleccionan elementos en listas o diccionarios.

In [ ]:
df[ 'Sexo' ]

Se pueden seleccionar varias columnas a la vez, poniendo sus nombres en una lista dentro de corchetes. Lo anterior significa que para seleccionar varias columnas hay que usar corchetes dobles.

In [ ]:
df[  ['Conocido_desconocido','Sexo']  ]

La primera columna que podemos observar en la tabla y carece de nombre, es el índice de las filas. 
Por default, pandas asigna números de identificación a cada una de las filas, empezando por el cero . 
Sin embargo, es posible especificar que columna es el indice cómo se vió al principio del tutorial.

Podemos seleccionar las filas por posición o por el nombren el indice.
Con `.iloc` se selecciona el número de posición de la fila, sin olvidar que la numeración empieza en 0. Por ejemplo, seleccionemos la novena fila.

In [ ]:
df.iloc[9]

De la misma manera, se pueden seleccionar varias filas usando una lista.

In [ ]:
df.iloc[  [9, 13, 62, 201]  ]

O se puede marcar una secuencia de posiciones como si fuera una lista

In [ ]:
df.iloc[  5:10  ]

Este comando también se puede usar para seleccionar un conjunto de filas y columnas específico usando la posición. Para hacerlo se utiliza el formato: `df.iloc[filas, columnas]`.

In [ ]:
df.iloc[ 9:15, [2,5,7] ]

Con `.loc` (sin la **i**) se selecciona la llave de la fila.

In [ ]:
df.loc[ 'BO_1968_00005' ]

Este comando también se puede usar para seleccionar un conjunto de filas y columnas específico por la llave o el nombre en el indice y las columnas. Para hacerlo se utiliza el formato: `df.loc[filas, columnas]`.

In [ ]:
df.loc[  ['BO_1968_00005','BO_1969_00005','BO_1970_00005','BO_1971_00005']  , 
         ['Conocido_desconocido','Sexo']  ]

## 4.c Únicos y conteo

Se pueden hacer varias cosas con una columna o `pd.Series`. 

Por ejemplo, se pueden determinar los valores únicos de la columna para entender la información contenida en ella con la función `.unique()`.

Por ejemplo, al revisar el profile se puede ver que hay múltiples instituciones de origen, así que obtendremos una lista de estas.

In [ ]:
df['Tipo_restos'].unique()

Si lo único que nos interesa saber es cuantos datos únicos hay, y no cuales son, se puede usar la función `.nunique()`.

In [ ]:
df['Tipo_restos'].nunique()

También, se puede contabilizar cuántas veces aparece cada uno de estos valores con `.value_counts()`. Esto permite saber cuantas veces aparece cada valor único. El resultado de la función es un dataframe.

In [ ]:
df['Tipo_restos'].value_counts()

Para ver estos resultados como porcentaje se puede usar el parámetro _normalize_. 


In [ ]:
df['Tipo_restos'].value_counts( normalize=True )

Es posible usar la función `.value_counts()` sobre varias columnas a la vez.
Esto regrese un dataframe con un `multi index`.

In [ ]:
df[ ['Sexo','Conocido_desconocido'] ].value_counts()

Es posible que en una columna o variable haya filas u observaciones donde no se registro el dato. 
Para incluir los NaN es necesario cambiar los parámetros de la función,usando el parámetro `dropna=False`.
Esto es de suma importancia al normalizar los datos, ya que el porcentaje de cada valor único cambiará dependiendo de si se toman, o no, en cuenta los NaN.

In [ ]:
df['Edad_transcrito'].value_counts(normalize=True ,  dropna=False, )

Como el resultado de la función `.value_counts()` es un dataframe podemos utilizar funciones propias de dataframe como `.head()` o slicing.

Por ejemplo, para ver las cinco instituciones que entregan más cadaveres obtendremos el conteo de valores con `.value_counts()` y después seleccionaremos los cinco primeros usando un slicing de lista.

In [ ]:
df['Procedencia_transcrito'].value_counts().head(5)

### Ejercicio 2

* ¿Cuáles son los cinco apellidos más comunes?
* ¿Cuántas veces aparece cada uno?
* ¿Qué porcentaje de la población tiene cada apellido? ¿Es lo mismo si normalizamos?
* ¿Cómo afectan los datos protegidos por razones legales estas cuentas?


## 4.d Ordenar

Además de poder seleccionar los datos muchas veces es muy importante poder ordenarlos. Para eso usaremos `.sort_values()`.

Para ordenar una sola columna:

In [ ]:
df['Nombres_propios'].sort_values()

Por default `.sort_values()` ordena en orden alfábetico o de menor a mayor, en caso de datos númericos. 
Para cambiar esto podemos usar el parámetro `ascending`. En el caso de los datos de texto los caracteres especiales van al principio. Por default los datos con `NaN` siempre van al final de la tabla.
Es importante destacar que los acentos pueden causar problemas al ordenar alfabeticamente.

In [ ]:
df['Nombres_propios'].sort_values( ascending=False )

Para ordenar todo el dataframe por una columna es necesario decir explicitamente que columna se va a usar.

Nota como en ambos casos se selecciona primero sobre que se va a trabajar, ya sea el dataframe completo (`df`) o una columna específica (`df[nombre_columna]`), después se dice que se va a hacer con el nombre de la función (`.sort_values()`) y finalmente se dan instrucciones específicas a la función de cómo hacerlo si es necesario poniendo los parametros dentro del parentesis.

In [ ]:
df.sort_values( 'Nombres_propios' )

Es posible ordenar por varias columnas especificando el nombre de las columnas en una lista y la forma de ordenarlas usando _ascending_.

In [ ]:
df.sort_values(  ['Sexo', 'Conocido_desconocido']  ,  ascending=[True,False]   )
df

## 4.e Subsetting

Es posible seleccionar los datos de acuerdo a una (o más) condición. 

Por ejemplo para seleccionar las filas con registros de mujeres buscamos las filas donde el 'Sexo' es 'Femenino' con la condición:

In [ ]:
df['Sexo'] == 'Femenino'

Esta condicion nos dice para cada fila si el valos es 'Femenino', si ponemos esto dentro de _.loc[]_ podemos usarlo para encontrar las filas con registros de mujeres.

In [ ]:
df.loc[ df['Sexo']=='Femenino' ]

Para seleccionar las filas que NO son registros de mujeres buscamos aquellos donde el sexo es diferenente (_!=_) a 'Femenino'

In [ ]:
df[  df['Sexo'] != 'Femenino'  ]

Si los datos son numéricos es posible usar operaciones como mayor, menor o igual para seleccionar con esas condiciones. Por ejemplo para ver los cadaveres menor de edad podriamos buscar todos las filas donde el valor de la edad es menor que 18.

**Nota** Al cargar los datos en la primera celda de código se limpio la celda para que solo incluyera valores numéricos.

In [ ]:
df[  df['Edad_transcrito']<18  ]

En varios casos podemos ver que no hay datos sobre el edad, lo cual se representa con _NaN_. Para ver las filas con valor _NaN_ se usa la función _isna()_.

Lo anterior puede representar un problema al momento del análisis, ya que significa que los datos son incompletos y generalmente es necesario procesarlos como veremos mas adelante. Es por esto que es importante revisar los faltantes.

In [ ]:
df[  df['Edad_transcrito'].isna()  ]

Es posible hacer lo contrario a la instrucción de selección agregando una negación al principio del comando de selección. 

Por ejemplo, para seleccionar las observaciones donde SI hay una edad, seleccionamos aquellas que no tienen Edad con isna() y luego las negamos. Recuerda que dos negaciones son una afirmación.

In [ ]:
df[ ~ df['Edad_transcrito'].isna() ]

A veces no queremos seleccionar las columnas que tienen un solo valor, sino varios. Por ejemplo, si queremos ver los restos que provienen de hospitales podemos generar una lista llamada 'hospitales' y despues los buscamos con el comando _.isin()_

In [ ]:
hospitales = ['HX', 'XO', 'HTB', 'VO', 'HTV', 'CR','HRL', 'HCM', 'MH-CR' ]
df[ df['Procedencia_estandar'].isin(hospitales) ]

Si queremos los registros que no provienen de hospitales, podemos usar el comando de negación `~` para obtener el conjunto complementario.

In [ ]:
df[ ~df['Procedencia_estandar'].isin(hospitales) ]

Se pueden usar varias condiciones al mismo tiempo.
Para combinar varias condiciones es necesario poner cada condición entre parentesis.
Las condiciones se unen usando los operadores booleanos:
* _&_ se cumplen ambas condiciones
* _|_ se cumple al menos una de las condiciones
* _~_ negación

Por ejemplo, para seleccionar los restos de mujeres menores de 18 años ponemos cada uno de los selectores de edades antes visto entre paréntesis y los unimos con _&_


In [ ]:
# df.loc[ (seleccionar_mujeres) & (seleccionar_menores_de_edad)] 
df.loc[  ( df['Sexo']=='Femenino' ) & ( df['Edad_transcrito']<18 ) ]

Es posible hacer combinaciones de selecciones bastante complicadas, sin embargo se recomienda guardarlas en un dataframe específico e ir haciendo las selecciones por pasos.

In [ ]:
df_vulnerables = df[ ~df['Edad_transcrito'].isna() ] #ignorar aquellos que no tienen edad
# seleccionar menores de edad 
s_menores = df_vulnerables['Edad_transcrito']<18
# seleccionar mujeres
s_mujeres = df_vulnerables['Sexo']=='Femenino'
# nota como seleccionamos sobre la primera seleccion y no el dataframe en general
df_vulnerables = df_vulnerables[ (s_menores) | (s_mujeres) ]
df_vulnerables #ver el resultado


Nota sobre fechas

Algunos tipos de datos permiten realizar operaciones especializadas. Por ejemplo con las fechas se pueden extraes el mes, día, día de la semana, hora, etc. Veremos esto más adelante pero dejamos un ejemplo de registros inhumados en viernes.

In [ ]:
df[ df['Fecha_estandar'].dt.day_name() == 'Friday' ]

### Ejercicio 3

* Escribe tres combinaciones de datos que te interesaria conocer y escribelas. Por ejemplo los restos recibidos en enero.
* Tratá de resolver estas combinaciones de datos usando pandas. En caso de no poder anota que datos u operaciones hacen falta.
* Escribe tres formas en las que te gustaría ordenar los datos.
* Obten las tablas ordenadas, en caso de no poder hacerlo indica que datos te harían falta.

## 4.f Guardar datos

Ya que tienen un subset, es posible guardarlo para utilizarlo mas adelante o abrirlo en otro programa como excel.

La nueva tabla debé de estar en una variable, en este caso usaremos `df_vulnerables`

In [ ]:
df_vulnerables

Después, decidiremos el nombre del archivo y lo guardaremos en una variable con el nombre del archivo de salida.

Finalmente, exportaremos la tabla en formato `.csv` con `.to_csv()`. Esta función tiene varios parametros, los cuales incluyen el nombre del archivo de salida, el incluir o no el indice, etc. Estas pueden ser vistas en la documentación.

In [ ]:
out_file = "data_clean/HBO_GruposVulnerables.csv"
df_vulnerables.to_csv(out_file)

**Gracias!**